In [1]:
from astropy.io import fits
import os
import subprocess
from astropy.table import Table
import numpy as np
from astropy.io import ascii
import time

import os
import numpy as np
from PIL import Image
import PIL.ImageOps    
from astropy.io import fits
from astropy.table import Table

import matplotlib.pyplot as plt
from astropy.visualization import astropy_mpl_style
import image_registration
from astropy.nddata import Cutout2D
from astropy import units as u
import matplotlib.image as mpimg

from __future__ import print_function
#import cv2
import numpy as np

# from photutils import DAOStarFinder
# from photutils import CircularAperture
# from photutils import aperture_photometry

from astropy.stats import sigma_clipped_stats

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

 
import pandas as pd
import pandasql as ps
plt.style.use(astropy_mpl_style)

#from regions.core import PixCoord
#from regions.shapes.circle import CirclePixelRegion
import scipy.misc
import time

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

In [2]:
def saveImData(imData,name):
    im = fits.PrimaryHDU(imData)
    hdub = fits.HDUList([im])
    hdub.writeto(name)

In [3]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [4]:
def getImageHeader(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].header
    return imageData

In [5]:
def assessQuality(output,sqlQuery,refPandasNew):
    pandasTable = Table.to_pandas(output)
    newdf = ps.sqldf(sqlQuery,locals())
    Nrecovered = len(newdf)
    Ntarget = len(refPandasNew)
    Ntot = len(pandasTable)
    return [Nrecovered,Ntarget,Ntot]

In [6]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='red',radius='10'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
            
    return fileName

In [24]:
def sextractor(imagename,valList,sextractoryDir='/home/mj1e16/sextractor/sextractor-master/config/',cataloguename='/home/mj1e16/sextractor/sextractor-master/config/moleTable.cat',tableDir='/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePosNewRegNormal//'):
    imTabName = imagename.replace('.fits','')
    loc1 = imTabName.rfind('/') + 1
    imTabName = imTabName[loc1:]+'{}_{}_{}'.format(valList[0],valList[1],valList[2])+'.cat'
    os.chdir(sextractoryDir)
    subprocess.call(['sex',imagename,'-c','default_1.sex'])
    assoc = Table.read(cataloguename,format='ascii.sextractor')
    subprocess.call(['cp',cataloguename,tableDir+imTabName])
    return assoc

In [26]:
def makeConfig(valList,tableName='moleTable.cat',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    cname = 'CATALOG_NAME'
    catName = tableName#'test1.cat'
    confName = 'default_1.sex'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    data = newData

    cname = 'PARAMETERS_NAME'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+confName,'w') as f:
        f.write(newData)
    return(confName,tableName,catName)

In [7]:
def findObjects(valList,image):
    makeConfig(valList)
    tabTot =  sextractor(image,valList)
    return tabTot

In [8]:
def saveSexTabasReg(table,regFileName='baseRegions.reg',regDirectory='/home/mj1e16/moleGazer/MoleGazer/regionDirectory/',aperture=15):
    xcoords = table['X_IMAGE'].tolist()
    ycoords = table['Y_IMAGE'].tolist()
    string = ''
    for x in range(len(xcoords)):
        string += 'circle({},{},{})\n'.format(xcoords[x],ycoords[x],aperture)
    with open(regDirectory+regFileName,'w') as f:
        f.write(string)

In [9]:
def transform16Bit(imageName,outImageName): 
    data = getImageData(imageName,0)
    hdu = fits.PrimaryHDU(data)
    hdu.data = np.int16(hdu.data)
    hdu.writeto(outImageName)

In [10]:
def showImageWithResults(valList,image,imageDir):
    tab = findObjects(valList,imageDir+image)
    saveSexTabasReg(tab)
    regions = makeDS9RegFile([tab],'/home/mj1e16/moleGazer/MoleGazer/regionDirectory/baseRegions','sex')
    print(imageDir+image)
    print(regions)
    subprocess.call(['ds9',imageDir+image,'-regions',regions])

In [11]:
def cropImage(image,ImDir):
    imData = getImageData(ImDir+image,0)
    ylength = imData.shape[0]
    cropHeight = int(ylength* 1./38.)
    imData[0:cropHeight] = 0
    hdu = fits.PrimaryHDU(imData)
    imageOut = image.replace('.fits','Cropped.fits')
    hdu.writeto(ImDir+'Cropped/'+imageOut)
    return imageOut

In [ ]:
# def normaliseImage(baseImage,targetImage): # needs analogue of which image is normalised
#     baseVal = 

In [12]:
def molePositions2Tab(molePositions,tollerance=5):
    with open(molePositions,'r') as f:
        data = f.readlines()

    ycoord = []
    xcoord = []
    starLocations = []
    for x in data:
        if 'circle(' in x:
            locator1 = '('
            locator2 = ','
            loc1 = x.index(locator1) +1
            #print(x)
            loc2 = x[loc1:].index(locator2) + loc1 +1
            loc3 = x[loc2:].index(locator2) + loc2
            xcoord.append(x[loc1:loc2-1])
            ycoord.append(x[loc2:loc3])
            #starLocations.append([float(xcoord),float(ycoord)])
    

    xUpper = [float(x) + tollerance for x in xcoord]
    xLower = [float(x) - tollerance for x in xcoord]
    yUpper = [float(x) + tollerance for x in ycoord]
    yLower = [float(x) - tollerance for x in ycoord]

    refDict = {'index':range(len(xcoord)),'xcoord':xcoord,'ycoord':ycoord,'xUpper':xUpper,'xLower':xLower,
              'yUpper':yUpper,'yLower':yLower}
    refPandasNew = pd.DataFrame(data=refDict)
    return refPandasNew

In [13]:
def readTable(imagename,valList,tableDir='/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/'):
    
    imTabName = imagename.replace('.fits','')
    loc1 = imTabName.rfind('/') + 1
    imTabName = imTabName[loc1:]+'{}_{}_{}'.format(valList[0],valList[1],valList[2])+'.cat'
    print(tableDir+imTabName)
    assoc = Table.read(tableDir+imTabName,format='ascii.sextractor')
    
    return assoc

In [ ]:
tab = readTable(sextractorReady[0],[valList[0][0],valList[1][0],valList[2][0]])

In [ ]:
tab

In [14]:
sqlcode = '''
SELECT *
FROM pandasTable
JOIN refPandasNew
ON pandasTable.X_IMAGE < refPandasNew.xUpper
AND pandasTable.X_IMAGE > refPandasNew.xLower
AND pandasTable.Y_IMAGE < refPandasNew.yUpper
AND pandasTable.Y_IMAGE > refPandasNew.yLower
'''

In [15]:
valList = [np.linspace(1,50,10),np.linspace(1,100,20),['default.conv','gauss_1.5_3x3.conv','gauss_2.0_3x3.conv','gauss_2.0_5x5.conv',
                                                    'gauss_2.5_5x5.conv','gauss_3.0_5x5.conv','gauss_3.0_7x7.conv',
                                                    'gauss_4.0_7x7.conv','gauss_5.0_9x9.conv','mexhat_1.5_5x5.conv',
                                                    'mexhat_2.0_7x7.conv','mexhat_2.5_7x7.conv','mexhat_3.0_9x9.conv',
                                                    'mexhat_4.0_9x9.conv','mexhat_5.0_11x11.conv','tophat_1.5_3x3.conv',
                                                    'tophat_2.0_3x3.conv','tophat_2.5_3x3.conv','tophat_3.0_3x3.conv',
                                                    'tophat_4.0_5x5.conv','tophat_5.0_5x5.conv']]

In [16]:
dirlist = os.listdir('/home/mj1e16/moleGazer/Photos/F/fitsImages/')
regList = [x for x in dirlist if '.reg' in x]
regList.sort()
imageList = [x for x in dirlist if '.fits' in x]
imageList.sort()

In [ ]:
# for x in imageList:
#     croppedIm = cropImage(x,'/home/mj1e16/moleGazer/Photos/F/fitsImages/')
#     moleImNewFormat = croppedIm.replace('.fits','16.fits')
#     transform16Bit('/home/mj1e16/moleGazer/Photos/F/fitsImages/Cropped/'+croppedIm,'/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/'+moleImNewFormat)
    

In [17]:
sextractorReady = os.listdir('/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/normalised/')
sextractorReady.sort()

In [21]:
regList = [x for x in regList if 'Labelled' not in x]
regList = [x for x in regList if 'temp' not in x]
print(regList)

['29005.reg', '29007.reg', '29009.reg', '29010.reg', '29011.reg', '29012.reg', '29013.reg', '29014.reg', '29015.reg', '29016.reg', '29017.reg', '29018.reg', '29019.reg', '29020.reg', '29021.reg', '29022.reg', '79940.reg', '79941.reg', '79942.reg', '79943.reg', '79944.reg', '79945.reg', '79946.reg', '79947.reg', '79948.reg', '79949.reg', '79950.reg', '79951.reg', '79952.reg', '79953.reg']


In [22]:
panTabs = []
for x in regList:
    panTabs.append(molePositions2Tab('/home/mj1e16/moleGazer/Photos/F/fitsImages/'+x))

In [ ]:
showImageWithResults([1,1,'default.conv'],'29005InvertedAverageCropped16.fits','/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/')

In [28]:
t0 = time.time()

#for regSize in range(6,22,2):

regSize = 10

panTabs = []
for x in regList:
    panTabs.append(molePositions2Tab('/home/mj1e16/moleGazer/Photos/F/fitsImages/'+x,tollerance=regSize))


for imNum in range(len(sextractorReady)):
    t1 = time.time()
    Nrecovered = []
    Ntarget = []
    Ntot = []
    settings = []

    fakeHeadMoleIM = '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/normalised/'+sextractorReady[imNum]
    refPandasNew = panTabs[imNum]

    for x in range(len(valList[0])):
        for y in range(len(valList[1])):
            for z in range(len(valList[2])):
                output = findObjects([valList[0][x],valList[1][y],valList[2][z]],fakeHeadMoleIM)
                #output = readTable(sextractorReady[imNum],[valList[0][x],valList[1][y],valList[2][z]])
                quality = assessQuality(output,sqlcode,refPandasNew)
                Nrecovered.append(quality[0])
                Ntarget.append(quality[1])
                Ntot.append(quality[2])
                settings.append([valList[0][x],valList[1][y],valList[2][z]])
    print(time.time() - t1)
    ogImNum = regList[imNum].replace('.reg','')
    with open('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePosNewRegNormal/{}_results_region_{}.py'.format(ogImNum,regSize),'w') as f:
        f.write('\nsettings = '+str(settings)+'\nNrecovered = '+str(Nrecovered)+'\nNtarget = '+str(Ntarget)+'\nNtot = '+str(Ntot))

print(time.time() -t0)

4081.27651191
4076.77532387
4048.00092697
4113.85894418
3976.73582697
4156.99982095
4016.40730715
3897.27100897
2203.59911513
2166.20539212
1874.06513405
1934.51835489
3855.64116287
3990.65757108
3766.03266597
3906.31814408
4099.0649178
3961.45555902
3800.22680712
4249.31539297
3918.01949
4090.35128284
2820.866009
2755.34970498
2828.79890299
3270.58515286
3823.6023581
3623.77369404
3851.91854692
4074.90303206
107233.108512


In [ ]:
t0 = time.time()



for imNum in range(len(sextractorReady)):
    t1 = time.time()
    Nrecovered = []
    Ntarget = []
    Ntot = []
    settings = []
    
    fakeHeadMoleIM = '/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/'+sextractorReady[imNum]
    refPandasNew = panTabs[imNum]
    
    for x in range(len(valList[0])):
        for y in range(len(valList[1])):
            for z in range(len(valList[2])):
                output = findObjects([valList[0][x],valList[1][y],valList[2][z]],fakeHeadMoleIM)
                quality = assessQuality(output,sqlcode,refPandasNew)
                Nrecovered.append(quality[0])
                Ntarget.append(quality[1])
                Ntot.append(quality[2])
                settings.append([valList[0][x],valList[1][y],valList[2][z]])
    print(time.time() - t1)
    ogImNum = regList[imNum].replace('.reg','')
    with open('/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/{}_results.py'.format(ogImNum),'w') as f:
        f.write('\nsettings = '+str(settings)+'\nNrecovered = '+str(Nrecovered)+'\nNtarget = '+str(Ntarget)+'\nNtot = '+str(Ntot))
        
print(time.time() -t0)

In [ ]:
testData = getImageData('/home/mj1e16/moleGazer/Photos/F/fitsImages/16bit/29005InvertedAverageCropped16.fits',0)

In [ ]:
np.std(testData)

In [ ]:
# command = ['ds9']
# for x in range(len(valList[0])):
#     for y in range(len(valList[1])):
#         command.extend([fakeHeadMoleIM,'-regions','sextest_{}_{}.reg'.format(x,y),'-scale','mode','zscale'])
# command.extend(['-frame','lock','image'])
# #print(command)
# subprocess.call(command)

In [ ]:
subprocess.call(['cp','/home/mj1e16/sextractor/sextractor-master/config/outputSextractorNew.py','/home/mj1e16/moleGazer/MoleGazer'])

In [ ]:
import outputSextractorNew as o2

In [ ]:
reload(outputSextractorNew)

In [ ]:
o2.Nrecovered

In [ ]:
outPutNecovered = o.Nrecovered
outPutNtarget = o.Ntarget
completeNess = [float(x)/float(y) if y != 0 else 0 for x, y in zip(outPutNecovered,outPutNtarget)]

In [ ]:
Nmax = max(o.Ntot)
accuracy = [float(x)/float(Nmax) for x in o.Ntot]

In [ ]:
compPerFilter = [[] for x in range(len(valList[2]))]
accPerfilter = [[] for x in range(len(valList[2]))]
settingsPerFilter = [[] for x in range(len(valList[2]))]
for x in range(len(valList[2])):
    for y in range(int(len(completeNess)/len(valList[2]))):
        #print((y*len(valList[2])+x))
        compPerFilter[x].append(completeNess[(y*len(valList[2])+x)])
        settingsPerFilter[x].append(o.settings[(y*len(valList[2])+x)])
        accPerfilter[x].append(accuracy[(y*len(valList[2])+x)])

In [ ]:
fullwhm = valList[0]
threshold = valList[1]

In [ ]:
(accPerfilter[19])

In [ ]:
def makeGraphfromFile(pyFile,valList,pyFileDirectory):
    
    subprocess.call(['cp',pyFileDirectory+pyFile,'./outputSextractorNew.py'])
    #pyFile = pyFile.replace('.py','')
    import outputSextractorNew as o
    
    outPutNecovered = o.Nrecovered
    outPutNtarget = o.Ntarget
    completeNess = [float(x)/float(y) if y != 0 else 0 for x, y in zip(outPutNecovered,outPutNtarget)]
    Nmax = max(o.Ntot)
    accuracy = [float(x)/float(Nmax) for x in o.Ntot]
    
    compPerFilter = [[] for x in range(len(valList[2]))]
    accPerfilter = [[] for x in range(len(valList[2]))]
    settingsPerFilter = [[] for x in range(len(valList[2]))]
    for x in range(len(valList[2])):
        for y in range(int(len(completeNess)/len(valList[2]))):
            #print((y*len(valList[2])+x))
            compPerFilter[x].append(completeNess[(y*len(valList[2])+x)])
            settingsPerFilter[x].append(o.settings[(y*len(valList[2])+x)])
            accPerfilter[x].append(accuracy[(y*len(valList[2])+x)])
            
    fullwhm = valList[0]
    threshold = valList[1]
    
    imageName = pyFile.replace('_results.py','')
    
    for y in range(len(valList[2])):
        accReformat = []
        compReformat = []
        for x in range(int(len(accPerfilter[y])/len(fullwhm))):
            accReformat.append(accPerfilter[y][x:x+len(fullwhm)])
            #print(accPerfilter[0][x:x+len(fullwhm)])
            compReformat.append(compPerFilter[y][x:x+len(fullwhm)])
            #totQualityReformat.append(tot[x:x+len(fullwhm)])

    #     fig = plt.figure(figsize=(15,10))
    #     ax = fig.gca(projection='3d')

        font = {'family' : 'serif',
                'weight' : 'normal',
                'size'   : 14}
        plt.rc('font', **font)
        plt.rc('axes',linewidth=1)
        #surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
        fig = plt.figure(figsize=(15,10))
        ax = fig.gca(projection='3d')
        X,Y = np.meshgrid(fullwhm,threshold)
        Z = np.array(accReformat)
        surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

        fig.colorbar(surf, shrink=0.5, aspect=5)
        #plt.gca().invert_zaxis()
        plt.xlabel('FWHM (Pixels)')
        plt.ylabel('Detection Threshold (Sigma)')
        ax.set_zlabel('Accuracy Score')
        plt.tight_layout()
        plt.savefig('/home/mj1e16/moleGazer/Photos/F/fitsImages/qualityCurves/acc_medium_filter_{}_image_{}_sextractor.png'.format(valList[2][y],imageName))
        plt.show()

    #     fig = plt.figure(figsize=(15,10))
    #     ax = fig.gca(projection='3d')

        font = {'family' : 'serif',
                'weight' : 'normal',
                'size'   : 14}
        plt.rc('font', **font)
        plt.rc('axes',linewidth=1)
        #surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
        fig = plt.figure(figsize=(15,10))
        ax = fig.gca(projection='3d')
        X,Y = np.meshgrid(fullwhm,threshold)
        Z = np.array(compReformat)
        surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

        fig.colorbar(surf, shrink=0.5, aspect=5)
        #plt.gca().invert_zaxis()
        plt.xlabel('FWHM (Pixels)')
        plt.ylabel('Detection Threshold (Sigma)')
        ax.set_zlabel('Completeness')
        plt.tight_layout()
        plt.savefig('/home/mj1e16/moleGazer/Photos/F/fitsImages/qualityCurves/comp_medium_filter_{}_image_{}_sextractor.png'.format(valList[2][y],imageName))
        plt.show()
    
        print(valList[2][y])

In [ ]:
#makeGraphfromFile('29005_results.py',valList,'/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/')

In [ ]:
outDir = '/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/'
dirList = os.listdir(outDir)
dirList = [x for x in dirList if '_results.py' in x]
print(dirList)

In [ ]:
# for result in dirList:
#     makeGraphfromFile(result,valList,'/home/mj1e16/moleGazer/Photos/F/fitsImages/simMolePos/')

In [ ]:
# accReformat = []
# compReformat = []
#totQualityReformat = []
for y in range(len(valList[2])):
    accReformat = []
    compReformat = []
    for x in range(int(len(accPerfilter[y])/len(fullwhm))):
        accReformat.append(accPerfilter[y][x:x+len(fullwhm)])
        #print(accPerfilter[0][x:x+len(fullwhm)])
        compReformat.append(compPerFilter[y][x:x+len(fullwhm)])
        #totQualityReformat.append(tot[x:x+len(fullwhm)])
        
#     fig = plt.figure(figsize=(15,10))
#     ax = fig.gca(projection='3d')

    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 14}
    plt.rc('font', **font)
    plt.rc('axes',linewidth=1)
    #surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
    fig = plt.figure(figsize=(15,10))
    ax = fig.gca(projection='3d')
    X,Y = np.meshgrid(fullwhm,threshold)
    Z = np.array(accReformat)
    surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

    fig.colorbar(surf, shrink=0.5, aspect=5)
    #plt.gca().invert_zaxis()
    plt.xlabel('FWHM (Pixels)')
    plt.ylabel('Detection Threshold (Sigma)')
    ax.set_zlabel('Accuracy Score')
    plt.tight_layout()
    plt.savefig('/home/mj1e16/moleGazer/MoleGazer/acc_medium_filter_{}_sextractor.png'.format(valList[2][y]))
    plt.show()
    
#     fig = plt.figure(figsize=(15,10))
#     ax = fig.gca(projection='3d')

    font = {'family' : 'serif',
            'weight' : 'normal',
            'size'   : 14}
    plt.rc('font', **font)
    plt.rc('axes',linewidth=1)
    #surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
    fig = plt.figure(figsize=(15,10))
    ax = fig.gca(projection='3d')
    X,Y = np.meshgrid(fullwhm,threshold)
    Z = np.array(compReformat)
    surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

    fig.colorbar(surf, shrink=0.5, aspect=5)
    #plt.gca().invert_zaxis()
    plt.xlabel('FWHM (Pixels)')
    plt.ylabel('Detection Threshold (Sigma)')
    ax.set_zlabel('Completeness')
    plt.tight_layout()
    plt.savefig('/home/mj1e16/moleGazer/MoleGazer/qualityCurves/comp_medium_filter_{}_sextractor.png'.format(valList[2][y]))
    plt.show()
    
    print(valList[2][y])

In [ ]:
print(len(fullwhm))
print(len(threshold))
print(len(accReformat))
print(len(accReformat[0]))

In [ ]:
accReformat

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.gca(projection='3d')

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('axes',linewidth=1)
#surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
fig = plt.figure(figsize=(15,10))
ax = fig.gca(projection='3d')
X,Y = np.meshgrid(fullwhm,threshold)
Z = np.array(accReformat)
surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

fig.colorbar(surf, shrink=0.5, aspect=5)
#plt.gca().invert_zaxis()
plt.xlabel('FWHM (Pixels)')
plt.ylabel('Detection Threshold (Sigma)')
ax.set_zlabel('Accuracy Score')
plt.tight_layout()
plt.savefig('/home/mj1e16/moleGazer/MoleGazer/accAverageImMeidumSim.png')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,10))
ax = fig.gca(projection='3d')

font = {'family' : 'serif',
        'weight' : 'normal',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('axes',linewidth=1)
#surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)
fig = plt.figure(figsize=(15,10))
ax = fig.gca(projection='3d')
X,Y = np.meshgrid(fullwhm,threshold)
Z = np.array(compReformat)
surf = ax.plot_surface(X,Y,Z,cmap='plasma',linewidth=0,antialiased=False)

fig.colorbar(surf, shrink=0.5, aspect=5)
#plt.gca().invert_zaxis()
plt.xlabel('FWHM (Pixels)')
plt.ylabel('Detection Threshold (Sigma)')
ax.set_zlabel('Completeness')
plt.tight_layout()
plt.savefig('/home/mj1e16/moleGazer/MoleGazer/compAverageImMeidumSim.png')
plt.show()